In [2]:
%load_ext autoreload
%autoreload 2

from aptrade.strategy import MarceloStrategy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import sys
sys.path.append('../src')
%load_ext autoreload
%autoreload 2

import os
from polygon import RESTClient
from dotenv import load_dotenv, dotenv_values
load_dotenv()

config = {
    **dotenv_values("../.env"),  # load shared development variables
    # **dotenv_values(".env.secret"),  # load sensitive variables
    # **os.environ,  # override loaded values with environment variables
}

print("Using Polygon API Key:", config.get("POLYGON_API_KEY"))
client = RESTClient(config.get("POLYGON_API_KEY")) # POLYGON_API_KEY is used

aggs = client.get_aggs(
    "VVOS",
    1,
    "minute",

    "2025-06-30",
    "2025-07-01",
)
print(aggs)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using Polygon API Key: XtppF2Lh0hKqwozkZdUpJpiaPYdPbDlO
[Agg(open=3.2, high=3.2, low=3.2, close=3.2, volume=361, vwap=3.2039, timestamp=1751279640000, transactions=9, otc=None), Agg(open=3.2, high=3.2, low=3.2, close=3.2, volume=191, vwap=3.2, timestamp=1751279880000, transactions=4, otc=None), Agg(open=3.35, high=3.35, low=3.35, close=3.35, volume=750, vwap=3.35, timestamp=1751283000000, transactions=2, otc=None), Agg(open=3.38, high=3.38, low=3.38, close=3.38, volume=524, vwap=3.3802, timestamp=1751285040000, transactions=2, otc=None), Agg(open=3.3312, high=3.3312, low=3.3312, close=3.3312, volume=100, vwap=3.3312, timestamp=1751285460000, transactions=1, otc=None), Agg(open=3.33, high=3.33, low=3.33, close=3.33, volume=300, vwap=3.33, timestamp=1751285520000, transactions=2, otc=None), Agg(open=3.2799, high=3.2799, low=3.2799, close=3.2799, volume=250, vwap=3.2776, timestamp=1751286240000, transa

In [10]:
# Convert the 't' (timestamp) column to date if present
import pandas as pd
df = pd.DataFrame(aggs)
df.head()

# Ensure 't' is present and convert to datetime
if 'timestamp' in df.columns:
    df['Date'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('Date', inplace=True)
    
print(df.index.tz)
if df.index.tz is None:
    df.index = df.index.tz_localize('UTC')
df.index = df.index.tz_convert('US/Eastern')
print(df.index.tz)

# Select and rename columns for Backtesting library
bt_cols = ["open", "high", "low", "close", "volume"]
df = df[bt_cols]
df.columns = ["Open", "High", "Low", "Close", "Volume"]
df.head()

None
US/Eastern


Open    High     Low   Close  Volume
Date                                                             
2025-06-30 06:34:00-04:00  3.2000  3.2000  3.2000  3.2000   361.0
2025-06-30 06:38:00-04:00  3.2000  3.2000  3.2000  3.2000   191.0
2025-06-30 07:30:00-04:00  3.3500  3.3500  3.3500  3.3500   750.0
2025-06-30 08:04:00-04:00  3.3800  3.3800  3.3800  3.3800   524.0
2025-06-30 08:11:00-04:00  3.3312  3.3312  3.3312  3.3312   100.0

In [11]:
from backtesting import Backtest, Strategy

# Fix: select columns as a list of separate strings
# testDF = df[["open", "high", "low", "close", "volume"]]

def per_share_commission(size, price):
    return abs(size) * 0.0005

# 3. create a backtest, pass in data and the strategy you want to run on the data
bt = Backtest(df, MarceloStrategy, cash=25_000, commission=per_share_commission, margin=0.25)

# run the strategy
stats = bt.run()
bt.plot()
print(stats)

stats['_trades'].to_csv('trade_details.csv', index=False)



New day detected: 2025-06-30
opening range high is 3.38
opening range low is 3.2
opening range percentage is 5.62%
New day detected: 2025-07-01
opening range high is 6.65
opening range low is 3.17
opening range percentage is 109.78%
Opening range percentage hit threshold: 109.78%
Both opening range percentage and pullback thresholds hit, proceeding with trade logic.
take_profit_price 3.15693
going short, position size 116 shares at planned price 4.5099, stop loss 6.65
closing out position


/home/vcaldas/aptrade/.venv/lib/python3.12/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


Start                     2025-06-30 06:34...
End                       2025-07-01 19:59...
Duration                      1 days 13:25:00
Exposure Time [%]                    53.28767
Equity Final [$]                    25066.004
Equity Peak [$]                     25107.822
Commissions [$]                         0.116
Return [%]                            0.26402
Buy & Hold Return [%]                19.05937
Return (Ann.) [%]                     39.4062
Volatility (Ann.) [%]                     NaN
CAGR [%]                             53.14132
Sharpe Ratio                              NaN
Sortino Ratio                             inf
Calmar Ratio                        102.76318
Alpha [%]                             0.32816
Beta                                 -0.00337
Max. Drawdown [%]                    -0.38347
Avg. Drawdown [%]                    -0.08992
Max. Drawdown Duration        0 days 09:41:00
Avg. Drawdown Duration        0 days 01:18:00
# Trades                          

In [50]:

from backtesting.test import SMA, GOOG
from backtesting.lib import crossover


class SmaCross(Strategy):
    n1 = 10
    n2 = 20

    def init(self):
        close = self.data.Close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()


bt = Backtest(df, MarceloStrategy,
              cash=10000, commission=.002,
              exclusive_orders=True)

output = bt.run()
bt.plot()

/tmp/ipykernel_48114/3565037952.py:27: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  output = bt.run()
/home/vcaldas/aptrade/.venv/lib/python3.12/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


GridPlot(id='p10315', ...)

In [52]:
from aptrade.strategy.opening_range_breakout import OpeningRangeBreakout
